#  Working With Datasets

Data is central to machine learning.  This tutorial introduces the `Dataset` class that DeepChem uses to store and manage data.  It provides simple but powerful tools for efficiently working with large amounts of data.  It also is designed to easily interact with other popular Python frameworks such as NumPy, Pandas, TensorFlow, and PyTorch.

## Colab

This tutorial and the rest in this sequence can be done in Google colab. If you'd like to open this notebook in colab, you can use the following link.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/deepchem/deepchem/blob/master/examples/tutorials/Working_With_Datasets.ipynb)



In [ ]:
!pip install --pre deepchem

We can now import the `deepchem` package to play with.

In [2]:
import deepchem as dc
dc.__version__

Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch_geometric'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. cannot import name 'DMPNN' from 'deepchem.models.torch_models' (C:\Users\cex\anaconda3\lib\site-packages\deepchem\models\torch_models\__init__.py)
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'pytorch_lightning'
Skipped loading some Jax models, missing a dependency. jax requires jaxlib to be installed. See https://github.com/google/jax#installation for installation instructions.


'2.7.1'

# Anatomy of a Dataset

In the last tutorial we loaded the Delaney dataset of molecular solubilities.  Let's load it again.

In [3]:
tasks, datasets, transformers = dc.molnet.load_delaney(featurizer='GraphConv')
train_dataset, valid_dataset, test_dataset = datasets

In [11]:
train_dataset.y

array([[ 3.90412938e-01],
       [ 9.04212762e-02],
       [-2.46434643e+00],
       [ 7.04920326e-01],
       [ 1.15974639e+00],
       [-3.53816788e-02],
       [-6.01494976e-01],
       [ 5.11377318e-01],
       [-1.46668908e-01],
       [-9.13583076e-01],
       [ 5.93633096e-01],
       [ 9.75880537e-01],
       [-9.40195240e-01],
       [ 1.31458080e+00],
       [ 1.08232919e+00],
       [ 6.37180273e-01],
       [-3.83759093e-01],
       [-1.05632104e+00],
       [ 1.29522650e+00],
       [ 9.90396262e-01],
       [ 9.04212762e-02],
       [ 2.98480010e-01],
       [ 4.58152991e-01],
       [ 5.50085920e-01],
       [ 4.21863677e-01],
       [ 1.54199383e+00],
       [-3.88597668e-01],
       [ 1.00007341e+00],
       [ 7.38790352e-01],
       [ 1.08232919e+00],
       [ 1.66779679e+00],
       [ 1.53322754e-01],
       [-1.51507483e-01],
       [-7.89288555e-02],
       [-9.35356665e-01],
       [-1.27889550e+00],
       [ 1.09684492e+00],
       [-7.56329383e-01],
       [ 9.9

In [ ]:
train_dataset.X[0]

We now have three Dataset objects: the training, validation, and test sets.  What information does each of them contain?  We can start to get an idea by printing out the string representation of one of them.

In [ ]:
print(test_dataset)

There's a lot of information there, so let's start at the beginning.  It begins with the label "DiskDataset".  Dataset is an abstract class.  It has a few subclasses that correspond to different ways of storing data.

- `DiskDataset` is a dataset that has been saved to disk.  The data is stored in a way that can be efficiently accessed, even if the total amount of data is far larger than your computer's memory.
- `NumpyDataset` is an in-memory dataset that holds all the data in NumPy arrays.  It is a useful tool when manipulating small to medium sized datasets that can fit entirely in memory.
- `ImageDataset` is a more specialized class that stores some or all of the data in image files on disk.  It is useful when working with models that have images as their inputs or outputs.

Now let's consider the contents of the Dataset.  Every Dataset stores a list of *samples*.  Very roughly speaking, a sample is a single data point.  In this case, each sample is a molecule.  In other datasets a sample might correspond to an experimental assay, a cell line, an image, or many other things.  For every sample the dataset stores the following information.

- The *features*, referred to as `X`.  This is the input that should be fed into a model to represent the sample.
- The *labels*, referred to as `y`.  This is the desired output from the model.  During training, it tries to make the model's output for each sample as close as possible to `y`.
- The *weights*, referred to as `w`.  This can be used to indicate that some data values are more important than others.  In later tutorials we will see examples of how this is useful.
- An *ID*, which is a unique identifier for the sample.  This can be anything as long as it is unique.  Sometimes it is just an integer index, but in this dataset the ID is a SMILES string describing the molecule.

Notice that `X`, `y`, and `w` all have 113 as the size of their first dimension.  That means this dataset contains 113 samples.

The final piece of information listed in the output is `task_names`.  Some datasets contain multiple pieces of information for each sample.  For example, if a sample represents a molecule, the dataset might record the results of several different experiments on that molecule.  This dataset has only a single task: "measured log solubility in mols per litre".  Also notice that `y` and `w` each have shape (113, 1).  The second dimension of these arrays usually matches the number of tasks.

# Accessing Data from a Dataset

There are many ways to access the data contained in a dataset.  The simplest is just to directly access the `X`, `y`, `w`, and `ids` properties.  Each of these returns the corresponding information as a NumPy array.

In [ ]:
test_dataset.y

This is a very easy way to access data, but you should be very careful about using it.  This requires the data for all samples to be loaded into memory at once.  That's fine for small datasets like this one, but for large datasets it could easily take more memory than you have.

A better approach is to iterate over the dataset.  That lets it load just a little data at a time, process it, then free the memory before loading the next bit.  You can use the `itersamples()` method to iterate over samples one at a time.

In [ ]:
for X, y, w, id in test_dataset.itersamples():
    print(y, id)

Most deep learning models can process a batch of multiple samples all at once.  You can use `iterbatches()` to iterate over batches of samples.

In [ ]:
for X, y, w, ids in test_dataset.iterbatches(batch_size=50):
    print(y.shape)

`iterbatches()` has other features that are useful when training models.  For example, `iterbatches(batch_size=100, epochs=10, deterministic=False)` will iterate over the complete dataset ten times, each time with the samples in a different random order.

Datasets can also expose data using the standard interfaces for TensorFlow and PyTorch.  To get a `tensorflow.data.Dataset`, call `make_tf_dataset()`.  To get a `torch.utils.data.IterableDataset`, call `make_pytorch_dataset()`.  See the API documentation for more details.

The final way of accessing data is `to_dataframe()`.  This copies the data into a Pandas `DataFrame`.  This requires storing all the data in memory at once, so you should only use it with small datasets.

In [12]:
test_dataset.to_dataframe()

,X,y,w,ids
0,<deepchem.feat.mol_graphs.ConvMol object at 0x...,-1.601145,1.0,c1cc2ccc3cccc4ccc(c1)c2c34
1,<deepchem.feat.mol_graphs.ConvMol object at 0x...,0.208483,1.0,Cc1cc(=O)[nH]c(=S)[nH]1
2,<deepchem.feat.mol_graphs.ConvMol object at 0x...,-0.016027,1.0,Oc1ccc(cc1)C2(OC(=O)c3ccccc23)c4ccc(O)cc4
3,<deepchem.feat.mol_graphs.ConvMol object at 0x...,-2.821917,1.0,c1ccc2c(c1)cc3ccc4cccc5ccc2c3c45
4,<deepchem.feat.mol_graphs.ConvMol object at 0x...,-0.528916,1.0,C1=Cc2cccc3cccc1c23
...,...,...,...,...
108,<deepchem.feat.mol_graphs.ConvMol object at 0x...,-1.656304,1.0,ClC4=C(Cl)C5(Cl)C3C1CC(C2OC12)C3C4(Cl)C5(Cl)Cl
109,<deepchem.feat.mol_graphs.ConvMol object at 0x...,0.743629,1.0,c1ccsc1
110,<deepchem.feat.mol_graphs.ConvMol object at 0x...,-2.420799,1.0,c1ccc2c(c1)ccc3c2ccc4c5ccccc5ccc43
111,<deepchem.feat.mol_graphs.ConvMol object at 0x...,-0.209570,1.0,Cc1occc1C(=O)Nc2ccccc2


# Creating Datasets

Now let's talk about how you can create your own datasets.  Creating a `NumpyDataset` is very simple: just pass the arrays containing the data to the constructor.  Let's create some random arrays, then wrap them in a NumpyDataset.

In [13]:
import numpy as np

X = np.random.random((10, 5))
y = np.random.random((10, 2))
dataset = dc.data.NumpyDataset(X=X, y=y)
print(dataset)

<NumpyDataset X.shape: (10, 5), y.shape: (10, 2), w.shape: (10, 1), ids: [0 1 2 3 4 5 6 7 8 9], task_names: [0 1]>


Notice that we did not specify weights or IDs.  These are optional, as is `y` for that matter.  Only `X` is required.  Since we left them out, it automatically built `w` and `ids` arrays for us, setting all weights to 1 and setting the IDs to integer indices.

In [14]:
dataset.to_dataframe()

,X1,X2,X3,X4,X5,y1,y2,w,ids
0,0.424143,0.276090,0.319609,0.983183,0.575875,0.814195,0.411448,1.0,0
1,0.012126,0.442284,0.377486,0.047734,0.711654,0.491575,0.981551,1.0,1
2,0.885062,0.171344,0.115304,0.245774,0.581921,0.346198,0.109353,1.0,2
3,0.476118,0.092328,0.702557,0.436578,0.597327,0.654779,0.524733,1.0,3
4,0.486684,0.876147,0.647389,0.179977,0.028378,0.600062,0.301585,1.0,4
5,0.651212,0.815168,0.318450,0.609154,0.581754,0.405770,0.007678,1.0,5
6,0.052058,0.179768,0.235275,0.431173,0.390472,0.946991,0.014824,1.0,6
7,0.106115,0.468635,0.406075,0.999428,0.673701,0.679976,0.810486,1.0,7
8,0.768017,0.706791,0.988571,0.667178,0.740848,0.669791,0.212123,1.0,8
9,0.219254,0.989193,0.697704,0.687757,0.160456,0.725308,0.079082,1.0,9


What about creating a DiskDataset?  If you have the data in NumPy arrays, you can call `DiskDataset.from_numpy()` to save it to disk.  Since this is just a tutorial, we will save it to a temporary directory.

In [15]:
import tempfile

with tempfile.TemporaryDirectory() as data_dir:
    disk_dataset = dc.data.DiskDataset.from_numpy(X=X, y=y, data_dir=data_dir)
    print(disk_dataset)

<DiskDataset X.shape: (10, 5), y.shape: (10, 2), w.shape: (10, 1), ids: [0 1 2 3 4 5 6 7 8 9], task_names: [0 1]>


What about larger datasets that can't fit in memory?  What if you have some huge files on disk containing data on hundreds of millions of molecules?  The process for creating a DiskDataset from them is slightly more involved.  Fortunately, DeepChem's `DataLoader` framework can automate most of the work for you.  That is a larger subject, so we will return to it in a later tutorial.

# Congratulations! Time to join the Community!

Congratulations on completing this tutorial notebook! If you enjoyed working through the tutorial, and want to continue working with DeepChem, we encourage you to finish the rest of the tutorials in this series. You can also help the DeepChem community in the following ways:

## Star DeepChem on [GitHub](https://github.com/deepchem/deepchem)
This helps build awareness of the DeepChem project and the tools for open source drug discovery that we're trying to build.

## Join the DeepChem Gitter
The DeepChem [Gitter](https://gitter.im/deepchem/Lobby) hosts a number of scientists, developers, and enthusiasts interested in deep learning for the life sciences. Join the conversation!